# Deep Featrues를 이용해서 이미지 분류하기

## graphlab import하기

graphlab을 사용하기 위해 import합시다.

In [2]:
import graphlab

# CIFAR-10 이미지 데이터 세트 로드하기

우리가 사용할 데이터는 CIFAR-10이라는 데이터 세트로서 컴퓨터 비전 분야에서 많이 사용하는 벤치 마크 데이터 세트입니다. 

https://www.cs.toronto.edu/~kriz/cifar.html


이 데이터 세트는 원래 10개 카테고리의 이미지들로 구성되어 있는데 본 수업을 위해서 4개 카테고리 - 'cat', 'bird'. automobile', 'dog' - 로 줄였고 학습 데이터와 테스트 데이터로 이미 나누어져 있습니다. 

아래 2개 폴더의 data를 로드해서 각각 학습 데이터와 테스트 데이터로 사용하고자 합니다.

    'image_train_data/'
    'image_test_data/'

In [3]:
!unzip -o ../data/image_test_data.zip -d ../data/
!unzip -o ../data/image_train_data.zip -d ../data/

Archive:  ../data/image_test_data.zip
  inflating: ../data/image_test_data/m_e16f5ffd2c088370.sidx  
  inflating: ../data/image_test_data/dir_archive.ini  
  inflating: ../data/image_test_data/m_e16f5ffd2c088370.frame_idx  
 extracting: ../data/image_test_data/objects.bin  
  inflating: ../data/image_test_data/m_e16f5ffd2c088370.0000  
Archive:  ../data/image_train_data.zip
  inflating: ../data/image_train_data/m_504edbda459b24ff.sidx  
  inflating: ../data/image_train_data/m_504edbda459b24ff.frame_idx  
  inflating: ../data/image_train_data/dir_archive.ini  
 extracting: ../data/image_train_data/objects.bin  
  inflating: ../data/image_train_data/m_504edbda459b24ff.0000  


In [4]:
test_data = graphlab.SFrame('../data/image_test_data/')
train_data = graphlab.SFrame('../data/image_train_data/')

This non-commercial license of GraphLab Create for academic use is assigned to cutty724@hanmail.net and will expire on December 24, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1520665588.log


In [5]:
# notebook에 그래프를 표시하도록 합니다.
graphlab.canvas.set_target('ipynb')

학습 데이터를 살펴 보세요.

In [6]:
train_data

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


이미지 id, 이미지 (바이너리값), 레이블, deep_features, 이미지 어레이 (픽셀값) 등을 확인할 수 있습니다.

학습 데이터의 길이를 확인해 봅시다.

In [7]:
len(train_data)

2005

겨우 2,005개 밖에 없네요. 학습이 쉽지 않아 보입니다.

## Quiz 

학습 데이터의 label별로 몇 개씩 데이터가 존재하는지 확인해 봅시다.

In [8]:
train_data['label'].show(view='Categorical')

대략 몇개씩 존재하나요?

## Quiz 

학습 데이터에서 4개의 레이블 중 가장 적은 수의 데이터를 가진 레이블은 무엇인가요?

In [9]:
#bird

# Image 데이터 출력해 보기

학습 데이터에 속한 이미지들을 출력해 보고자 합니다.

먼저 학습 데이터의 'image' 컬럼의 유형을 확인하세요.

In [10]:
type(train_data['image'])

graphlab.data_structures.sarray.SArray

SArray가 맞나요?

학습 데이터의 'image' 컬럼의 첫번째 데이터를 출력해 보세요.

In [11]:
train_data['image'][0]

Height: 32px
Width: 32px
Channels: 3

우리가 확인할 수 있는 형태의 데이터로 표시되지 않고 단순히 높이, 폭, 채널 수 등이 표시되네요.

이제 학습 데이터의 'image' 컬럼의 첫번째 줄의 데이터의 유형을 확인하세요

In [12]:
type(train_data['image'][0])

graphlab.data_structures.image.Image

Image가 맞나요? 그렇다면 아래 명령을 이용해서 Image 데이터가 들어있는 SArray에 속한 이미지들을 출력할 수 있습니다.

    SArray.show()
    
주의: Image 유형은 바로 출력이 안되고 SArray만 show 메소드를 이용해서 출력이 가능합니다. 따라서 한개의 이미지를 출력할 때에도 Slice notation을 이용한 SArray가 필요합니다. 

## Quiz 1

학습 데이터의 첫번째 줄의 이미지를 출력해 봅시다.

In [13]:
train_data['image'][0:1].show()

이제 학습 데이터의 'image' 컬럼 전체에 대해서 show메소드를 실행해 봅시다.

In [14]:
train_data['image'].show()

개수가 20개를 넘어가는 경우 처음 20개의 이미지만 출력됩니다.

## (참고) 이미지 크기를 조절하는 법

아래 메소드를 이용하면 이미지 크기를 조절할 수 있습니다. 폭과 높이를 조절한 이미지의 SArray를 리턴합니다.

    graphlab.image_analaysis.resize(SArray, 폭, 높이)

In [15]:
# 예제
graphlab.image_analysis.resize(train_data[0:5]['image'],100,100).show()

아쉽게도 원본 이미지의 해상도가 낮아서 확대를 해도 그다지 좋은 결과를 보여주지 않습니다. 이미지가 잘 보이지 않는 경우 해당 코드를 사용해서 확대해 보세요.

## Quiz

학습 데이터의 20번째 이미지부터 30번째 이미지를 출력해 보세요.

In [16]:
train_data['image'][19:29].show()

개, 개, 고양이, 자동차, .... , 고양이, 새 순으로 10개의 이미지가 출력되어야 합니다.

## Quiz

학습 데이터 중에서 뒤에서 10개의 이미지를 출력해 보세요.

In [17]:
train_data['image'][-10::].show()

고양이, 고양이, 개, 새, ..., 고양이, 고양이 순으로 10개의 이미지가 출력되어야 합니다.

## image_array 컬럼 살펴 보기

image_array 컬럼의 맨 첫번째 행을 출력해 봅시다.

In [18]:
train_data['image_array'][0]

array('d', [73.0, 77.0, 58.0, 71.0, 68.0, 50.0, 77.0, 69.0, 44.0, 120.0, 116.0, 83.0, 125.0, 120.0, 90.0, 155.0, 148.0, 117.0, 147.0, 152.0, 106.0, 133.0, 136.0, 92.0, 112.0, 101.0, 75.0, 100.0, 84.0, 68.0, 100.0, 78.0, 74.0, 69.0, 49.0, 51.0, 22.0, 16.0, 13.0, 35.0, 27.0, 24.0, 52.0, 40.0, 40.0, 70.0, 55.0, 54.0, 117.0, 97.0, 89.0, 122.0, 101.0, 99.0, 103.0, 85.0, 88.0, 88.0, 74.0, 73.0, 68.0, 59.0, 52.0, 72.0, 65.0, 52.0, 82.0, 73.0, 60.0, 84.0, 72.0, 68.0, 84.0, 67.0, 73.0, 77.0, 56.0, 65.0, 93.0, 75.0, 81.0, 117.0, 102.0, 98.0, 174.0, 171.0, 146.0, 183.0, 210.0, 149.0, 180.0, 214.0, 143.0, 185.0, 225.0, 144.0, 73.0, 74.0, 52.0, 75.0, 68.0, 55.0, 78.0, 64.0, 50.0, 103.0, 90.0, 64.0, 80.0, 65.0, 35.0, 127.0, 110.0, 80.0, 136.0, 127.0, 98.0, 141.0, 134.0, 109.0, 129.0, 113.0, 92.0, 115.0, 97.0, 72.0, 111.0, 91.0, 61.0, 115.0, 96.0, 67.0, 94.0, 83.0, 65.0, 92.0, 78.0, 65.0, 78.0, 60.0, 53.0, 89.0, 68.0, 64.0, 123.0, 99.0, 92.0, 117.0, 93.0, 92.0, 118.0, 97.0, 102.0, 121.0, 103.0, 103.0

숫자들이 리스트 안에 잔뜩 들어 있습니다. 

image_array 컬럼의 맨 첫번째 행의 길이를 확인하세요.

In [19]:
len(train_data['image_array'][0])

3072

길이가 3072인 것을 확인할 수 있습니다. 이는 32 x 32로 이루어진 RGB 이미지 (1,024개 픽셀, 3채널)의 데이터입니다. 32 x 32 x 3 = 3,072가 됩니다. 3개 채널의 값을 죽 이어서 하나의 리스트에 넣었습니다.


## 생각해 볼 문제

이러한 모델링은 어떤 문제를 가질까요? 서로의 상관 관계를 파악하는 데 유리한 모델일까요?

# Logistic Classifier를 이용한 모델 만들기

image_array에 저장된 픽셀들의 값을 모두 개별 피쳐로 이용해서 logistic classifier 모델을 만들고자 합니다.

    graphlab.logistic_classifier.create(학습데이터, target=레이블컬럼이름, features=사용하고자하는피쳐리스트, validation_set=None)
    
이 모델을 **raw_pixel_model**로 저장합시다.

참고: 학습 데이터의 양이 작아서 별도의 검증 세트를 사용하지 않습니다.

In [20]:
raw_pixel_model = graphlab.logistic_classifier.create(train_data, target='label', features=['image_array'], validation_set=None)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 2005

Number of classes           : 4

...

| 9         | 15       | 1.000000  | 21.960589    | 0.509227          |

| 10        | 16       | 1.000000  | 24.405625    | 0.512219          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

위의 메시지 중에서 자세히 볼 부분이 몇가지 있습니다.


**다음 메시지는 무슨 뜻일까요?**

    Number of feature columns   : 1
    
    Number of unpacked features : 3072
    
    Number of coefficients    : 9219

**그리고 경고 메시지를 잘 살펴 봅시다. 무슨 뜻일까요?**

    WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

**마지막 메시지도 잘 살펴 봅시다. 무슨 뜻일까요?**

    This model may not be optimal. To improve it, consider increasing `max_iterations`.

# 모델 검증하기

학습 데이터의 첫 3개 이미지를 눈으로 확인합시다. 어떤 이미지들인가요?

In [21]:
train_data['image'][:3].show()

각각 새, 고양이, 고양이 사진이 맞나요? 학습 데이터에 저장되어 있는 레이블을 확인해 봅시다.

In [22]:
train_data['label'][:3]

dtype: str
Rows: 3
['bird', 'cat', 'cat']

우리가 눈으로 확인한 것과 동일한 레이블이 저장되어 있습니다.

이제 이 이미지들을 학습한 모델을 이용해서 예측해 봅시다. predict 메소드를 사용하면 됩니다.

    model.predict(SArray)

In [23]:
raw_pixel_model.predict(train_data[:3])

dtype: str
Rows: 3
['automobile', 'dog', 'cat']

나온 결과가 레이블과 일치하나요? (아마 하나만 맞췄을 겁니다... ㅠ.ㅜ)

## 노멀라이즈 적용하기


학습 데이터의 image_array의 첫줄을 다시 한번 살펴 봅시다.

In [24]:
train_data['image_array'][0]

array('d', [73.0, 77.0, 58.0, 71.0, 68.0, 50.0, 77.0, 69.0, 44.0, 120.0, 116.0, 83.0, 125.0, 120.0, 90.0, 155.0, 148.0, 117.0, 147.0, 152.0, 106.0, 133.0, 136.0, 92.0, 112.0, 101.0, 75.0, 100.0, 84.0, 68.0, 100.0, 78.0, 74.0, 69.0, 49.0, 51.0, 22.0, 16.0, 13.0, 35.0, 27.0, 24.0, 52.0, 40.0, 40.0, 70.0, 55.0, 54.0, 117.0, 97.0, 89.0, 122.0, 101.0, 99.0, 103.0, 85.0, 88.0, 88.0, 74.0, 73.0, 68.0, 59.0, 52.0, 72.0, 65.0, 52.0, 82.0, 73.0, 60.0, 84.0, 72.0, 68.0, 84.0, 67.0, 73.0, 77.0, 56.0, 65.0, 93.0, 75.0, 81.0, 117.0, 102.0, 98.0, 174.0, 171.0, 146.0, 183.0, 210.0, 149.0, 180.0, 214.0, 143.0, 185.0, 225.0, 144.0, 73.0, 74.0, 52.0, 75.0, 68.0, 55.0, 78.0, 64.0, 50.0, 103.0, 90.0, 64.0, 80.0, 65.0, 35.0, 127.0, 110.0, 80.0, 136.0, 127.0, 98.0, 141.0, 134.0, 109.0, 129.0, 113.0, 92.0, 115.0, 97.0, 72.0, 111.0, 91.0, 61.0, 115.0, 96.0, 67.0, 94.0, 83.0, 65.0, 92.0, 78.0, 65.0, 78.0, 60.0, 53.0, 89.0, 68.0, 64.0, 123.0, 99.0, 92.0, 117.0, 93.0, 92.0, 118.0, 97.0, 102.0, 121.0, 103.0, 103.0

값이 0~255 사이에서 상당히 큰 널뛰기를 하고 있는 것을 확인할 수 있습니다. 우리는 RGB 이미지 픽셀의 값이 0 ~ 255 사이의 값을 가진다는 것을 알고 있습니다. 

**이 점을 고려할 때 무엇을 할 수 있을까요?**

## Quiz

image_array 안의 값들을 normalize해서 0~1 사이로 만들어 봅시다. 이를 위해 **normalize_pixel**이라는 함수를 만들어 봅시다.

인자로는 array를 받고 안의 값들을 0과 1사이의 값으로 변환한 다음에 다시 리스트에 넣어 결과값으로 반환하고자 합니다. 

**주의: 여기서 array와 list의 차이에 주목하세요.**

In [25]:
def normalize_pixel(array):
    new_list = []
    for i in array:
        new_list.append(i/255)
    return new_list

In [26]:
normalize_pixel(train_data['image_array'][0])

[0.28627450980392155,
 0.30196078431372547,
 0.22745098039215686,
 0.2784313725490196,
 0.26666666666666666,
 0.19607843137254902,
 0.30196078431372547,
 0.27058823529411763,
 0.17254901960784313,
 0.47058823529411764,
 0.4549019607843137,
 0.3254901960784314,
 0.49019607843137253,
 0.47058823529411764,
 0.35294117647058826,
 0.6078431372549019,
 0.5803921568627451,
 0.4588235294117647,
 0.5764705882352941,
 0.596078431372549,
 0.41568627450980394,
 0.5215686274509804,
 0.5333333333333333,
 0.3607843137254902,
 0.4392156862745098,
 0.396078431372549,
 0.29411764705882354,
 0.39215686274509803,
 0.32941176470588235,
 0.26666666666666666,
 0.39215686274509803,
 0.3058823529411765,
 0.2901960784313726,
 0.27058823529411763,
 0.19215686274509805,
 0.2,
 0.08627450980392157,
 0.06274509803921569,
 0.050980392156862744,
 0.13725490196078433,
 0.10588235294117647,
 0.09411764705882353,
 0.20392156862745098,
 0.1568627450980392,
 0.1568627450980392,
 0.27450980392156865,
 0.21568627450980393,


## Quiz

위에서 만든 함수를 이용해서 image_array에 들어 있는 픽셀 데이터 전체를 normalize한 후에 **image_train**에 새로운 컬럼 **image_array_norm**에 저장하고자 합니다. 어떻게 해야 할까요?

**주의: 먼저 apply를 적용해서 함수가 잘 적용되는지 확인한 후에 해당 결과를 새로운 컬럼에 저장하세요.**

In [27]:
train_data['image_array'][0:1].apply(normalize_pixel)

dtype: array
Rows: 1
[array('d', [0.28627450980392155, 0.30196078431372547, 0.22745098039215686, 0.2784313725490196, 0.26666666666666666, 0.19607843137254902, 0.30196078431372547, 0.27058823529411763, 0.17254901960784313, 0.47058823529411764, 0.4549019607843137, 0.3254901960784314, 0.49019607843137253, 0.47058823529411764, 0.35294117647058826, 0.6078431372549019, 0.5803921568627451, 0.4588235294117647, 0.5764705882352941, 0.596078431372549, 0.41568627450980394, 0.5215686274509804, 0.5333333333333333, 0.3607843137254902, 0.4392156862745098, 0.396078431372549, 0.29411764705882354, 0.39215686274509803, 0.32941176470588235, 0.26666666666666666, 0.39215686274509803, 0.3058823529411765, 0.2901960784313726, 0.27058823529411763, 0.19215686274509805, 0.2, 0.08627450980392157, 0.06274509803921569, 0.050980392156862744, 0.13725490196078433, 0.10588235294117647, 0.09411764705882353, 0.20392156862745098, 0.1568627450980392, 0.1568627450980392, 0.27450980392156865, 0.21568627450980393, 0.21176470588

In [28]:
train_data['image_array_norm'] = train_data['image_array'].apply(normalize_pixel)

image_train에 새로운 컬럼이 잘 만들어 졌는지, 데이터는 0과 1사이인지 확인해 봅시다.

In [29]:
train_data['image_array_norm'][0]

array('d', [0.28627450980392155, 0.30196078431372547, 0.22745098039215686, 0.2784313725490196, 0.26666666666666666, 0.19607843137254902, 0.30196078431372547, 0.27058823529411763, 0.17254901960784313, 0.47058823529411764, 0.4549019607843137, 0.3254901960784314, 0.49019607843137253, 0.47058823529411764, 0.35294117647058826, 0.6078431372549019, 0.5803921568627451, 0.4588235294117647, 0.5764705882352941, 0.596078431372549, 0.41568627450980394, 0.5215686274509804, 0.5333333333333333, 0.3607843137254902, 0.4392156862745098, 0.396078431372549, 0.29411764705882354, 0.39215686274509803, 0.32941176470588235, 0.26666666666666666, 0.39215686274509803, 0.3058823529411765, 0.2901960784313726, 0.27058823529411763, 0.19215686274509805, 0.2, 0.08627450980392157, 0.06274509803921569, 0.050980392156862744, 0.13725490196078433, 0.10588235294117647, 0.09411764705882353, 0.20392156862745098, 0.1568627450980392, 0.1568627450980392, 0.27450980392156865, 0.21568627450980393, 0.21176470588235294, 0.458823529411

### 새로운 데이터로 새로운 모델 학습하기

위와 마찬가지로 logistic_classifier를 이용해서 새로운 피쳐 - normalized 픽셀데이터 - 로 학습해 봅시다.

In [30]:
norm_pixel_model = graphlab.logistic_classifier.create(train_data, target='label', features=['image_array_norm'], validation_set=None)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 2005

Number of classes           : 4

...

| 9         | 15       | 1.000000  | 28.130407    | 0.509227          |

| 10        | 16       | 1.000000  | 30.624048    | 0.512219          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

이전의 결과 (0.512219)와 비교해 보면 **학습 데이터에 대한 정확도가 변하지 않았다는 것을 확인**할 수 있습니다. 

왜일까요? 아마도 **graphlab이 이 정도 normalize는 자동으로 하는 것으로 추정됩니다.**

다시 한번 학습데이터의 첫 3개 이미지에 대해서 새로운 모델을 이용해서 예측을 해 봅시다.

In [31]:
## 원래 학습데이터의 레이블
train_data[0:3]['label']

dtype: str
Rows: 3
['bird', 'cat', 'cat']

In [32]:
norm_pixel_model.predict(train_data[0:3])

dtype: str
Rows: 3
['automobile', 'dog', 'cat']

새로운 모델에서도 하나만 맞았네요. normalize를 안한 모델과 차이가 없습니다. ㅠ.ㅜ

어떻게 해야 할까요? (힌트: 아까 학습 시에 나왔던 메시지들에 따라 하나씩 개선해 봅시다.)

### max_iterations 추가하기

이번에는 지난 모델 학습시에 맨 마지막에 있었던 메시지에 주목해 봅시다.

    TERMINATED: Iteration limit reached.
    
    This model may not be optimal. To improve it, consider increasing `max_iterations`.


normalize한 모델에 더해서 **max_iterations**를 20으로 지정해 봅시다.

**주의: 이하에서 max_iterations 값을 더 증가시킬 예정이라서 구별을 위해 모델 이름에 20을 포함해서 만드세요.**

In [33]:
norm_pixel_model_20 = graphlab.logistic_classifier.create(train_data, target='label', features=['image_array_norm'], validation_set=None, max_iterations=20)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 2005

Number of classes           : 4

...

| 19        | 27       | 1.000000  | 54.189199    | 0.571072          |

| 20        | 28       | 1.000000  | 56.406402    | 0.577556          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [34]:
norm_pixel_model_20.evaluate(train_data)['accuracy']

0.5775561097256858

이전 (0.512219)보다 결과 (0.577556)가 조금 더 나아졌습니다!

이번에는 50을 사용해 봅시다.

In [35]:
norm_pixel_model_50 = graphlab.logistic_classifier.create(train_data, target='label', features=['image_array_norm'], validation_set=None, max_iterations=50)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 2005

Number of classes           : 4

...

| 49        | 57       | 1.000000  | 114.150370   | 0.634414          |

| 50        | 58       | 1.000000  | 117.166184   | 0.625436          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [36]:
norm_pixel_model_50.evaluate(train_data)['accuracy']

0.6254364089775561

이전 (0.577556)보다 결과 (0.625436)가 조금 더 나아졌습니다!

이번에는 100을 사용해 봅시다.

In [37]:
norm_pixel_model_100 = graphlab.logistic_classifier.create(train_data, target='label', features=['image_array_norm'], validation_set=None, max_iterations=100)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 2005

Number of classes           : 4

...

| 99        | 112      | 1.000000  | 231.378219   | 0.732668          |

| 100       | 113      | 1.000000  | 234.681246   | 0.732170          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [38]:
norm_pixel_model_100.evaluate(train_data)['accuracy']

0.7321695760598503

이전 (0.625436)보다 결과 (0.732170)가 훨씬 더 나아졌습니다!

이렇게 계속 값을 올려야 할까요7?

참고로 200회 실행하면 정확도가 0.850374로 올라가는 것을 확인할 수 있습니다.

## 이미지 테스트 세트를 이용해서 모델 검증해 보기


더 이터레이션을 올려 보기 전에 이미지 테스트 세트를 이용해서 예측이 잘 되는지 확인해 봅시다.

### image_test 픽셀데이터 normalize하기

먼저 image_test 세트의 픽셀 데이터도 동일한 조건으로 normalize해야 합니다. 

## Quiz 

**왜일까요?**

위에서 만든 함수로 normalize해 봅시다.

In [39]:
test_data['image_array_norm'] = test_data['image_array'].apply(normalize_pixel)

제대로 적용되었는지 확인해 봅시다.

In [40]:
test_data['image_array_norm'][0]

array('d', [0.6196078431372549, 0.4392156862745098, 0.19215686274509805, 0.6235294117647059, 0.43529411764705883, 0.1843137254901961, 0.6470588235294118, 0.4549019607843137, 0.2, 0.6509803921568628, 0.4627450980392157, 0.20784313725490197, 0.6274509803921569, 0.4392156862745098, 0.1803921568627451, 0.611764705882353, 0.42745098039215684, 0.1607843137254902, 0.6352941176470588, 0.45098039215686275, 0.1843137254901961, 0.6235294117647059, 0.44313725490196076, 0.17647058823529413, 0.6196078431372549, 0.43529411764705883, 0.17254901960784313, 0.6235294117647059, 0.44313725490196076, 0.1607843137254902, 0.6313725490196078, 0.4549019607843137, 0.1607843137254902, 0.6274509803921569, 0.43529411764705883, 0.20392156862745098, 0.6313725490196078, 0.43529411764705883, 0.19215686274509805, 0.6509803921568628, 0.4588235294117647, 0.1607843137254902, 0.6627450980392157, 0.4588235294117647, 0.17647058823529413, 0.6666666666666666, 0.4666666666666667, 0.17254901960784313, 0.6549019607843137, 0.458823

### image_test로 모델 검증하기

## Quiz

raw_pixel_model, 노멀라이즈를 적용한 모델의 정확도는 각각 얼마인가요? 그리고 노멀라이즈를 적용한 모델의 iteration을 20, 50, 100회으로 증가시킨 결과는 각각 얼마인가요? 

총 5개의 모델의 정확도를 비교하세요.

    modle.????(테스트데이터)
    
어떤 메소드를 써야 할까요?

In [41]:
raw_pixel_model.evaluate(test_data)['accuracy']

0.477

In [42]:
norm_pixel_model.evaluate(test_data)['accuracy']

0.477

In [43]:
norm_pixel_model_20.evaluate(test_data)['accuracy']

0.49925

In [44]:
norm_pixel_model_50.evaluate(test_data)['accuracy']

0.451

In [45]:
norm_pixel_model_100.evaluate(test_data)['accuracy']

0.46675

## Quiz

노멀라이즈 데이터를 적용한 모델의 iteration을 높여 가니 20회에서 최고점을 찍고 이후로는 점점 정확도가 떨어지는 것을 확인할 수 있습니다. 왜일까요?

# 암울한 결과....

max_iterations 값을 높여 봐도 테스트 세트에서 정확도가 올라가지 않는다는 것을 확인했습니다. 

아무리 노력을 해도 모델의 정확도가 50%이하로 밖에 나오질 않습니다. ㅠ.ㅜ

이제 어떻게 해야 할까요?

## deep features를 이용해서 모델을 개선할 수 있을까요?

학습 데이터가 2,005개 밖에 없기 때문에 deep learning 모델을 학습시킬 수는 없습니다. 그래서 전체 ImageNet 데이터 세트를 통해 학습된 deep features를 이용해서 우리가 가지고 있는 작은 학습 데이터를 학습시켜 간단한 모델을 만들고자 합니다. 이를 **transfer learning**이라고 부릅니다.

## 미리 생성된 모델을 이용하여 학습 데이터에 대한 deep features를 계산하기

이하의 2줄은 다른 데이터로 미리 학습된 deep learning 모델을 우리 학습 데이터에 적용하여 deep features를 생성하는 코드입니다. (하지만 시간이 너무 오래 걸리기 때문에 이하의 과정은 생략하고 학습 데이터에 미리 저장해 둔 deep_features 컬럼의 데이터를 사용합니다.)

In [46]:
# deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
# image_train['deep_features'] = deep_learning_model.extract_features(image_train)

학습 데이터에서 deep_features 컬럼의 내용을 살펴 봅시다. 이 학습 데이터에 미리 계산된 deep features값이 저장되어 있는 것을 확인할 수 있습니다.

In [47]:
train_data['deep_features'].head()

dtype: array
Rows: 10
[array('d', [0.24287176132202148, 1.0954537391662598, 0.0, 0.39362990856170654, 0.0, 0.0, 11.894915580749512, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5788496136665344, 0.4954667389392853, 2.5141289234161377, 0.0, 1.5180106163024902, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5813961029052734, 0.0, 0.0, 2.595609426498413, 2.7079553604125977, 0.0, 0.0, 0.0, 0.8509902954101562, 0.0, 0.7203489542007446, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2700355052947998, 0.0, 0.0, 0.0, 0.0, 0.08592796325683594, 0.0, 0.7010231018066406, 0.0, 0.0, 0.0, 0.0, 0.024805665016174316, 0.0, 0.0, 0.17549043893814087, 0.0, 0.0, 0.0, 0.0, 0.0, 2.392784595489502, 0.0, 0.0, 4.471865653991699, 0.0, 1.6358323097229004, 0.0, 4.417484760284424, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4117904901504517, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1247677206993103, 0.0, 0.0, 0.8957164287567139, 0.0, 0.0, 0.3334987759590149, 0.0, 0.0, 0.2078730463981628

이 deep features를 이용해서 logistic_classifier 모델을 만들어 봅시다.

이 모델을 **deep_feature_model**에 저장합시다.

In [48]:
deep_feature_model = graphlab.logistic_classifier.create(train_data, target='label', features=['deep_features'], validation_set=None)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

WARNING: Detected extremely low variance for feature(s) 'deep_features' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

Number of examples          : 2005

...

| 9         | 16       | 0.250000  | 36.897003    | 0.839900          |

| 10        | 17       | 0.250000  | 40.052572    | 0.869825          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

주목해야 할 메시지가 있습니다.

    Number of feature columns   : 1
    
    Number of unpacked features : 4096
    
    Number of coefficients    : 12291
    
deep features에 속한 피쳐의 수가 4,096개입니다.

이전과 마찬가지로 max_iterations를 높여 볼 수 있겠네요.

    TERMINATED: Iteration limit reached.
    
    This model may not be optimal. To improve it, consider increasing `max_iterations`.

deep features를 이용하니 정확도가 무려 86.98%로 올라갔네요. 하지만 테스트 데이터에 적용해 보기 전에는 확신할 수 없습니다.

## 테스트 데이터에 deep features model 적용하기

테스트 데이터의 처음 3개 이미지를 화면에 출력해 봅시다.

In [49]:
test_data['image'][:3].show()

이 3개 이미지를 deep_feature_model로 예측해 봅시다.

In [50]:
deep_feature_model.predict(test_data[:3])

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

3개 이미지 모두에 대해서 정확하게 레이블을 맞췄나요? 와우!!! 뭔가 가능성이 보입니다.

## deep features model의 정확도를 계산하기

이제 전체 테스트 데이터를 이용해서 이 모델을 검증해 봅시다.

    model.evalaute(데이터)

In [51]:
deep_feature_model.evaluate(test_data)['accuracy']

0.792

deep_feature_model이 테스트 세트에 대해서 **78.575%의 정확도**를 보이는 것을 확인할 수 있습니다. 브라보!!!

## Quiz

이제 무엇을 더해 볼 수 있을까요? 다양한 방법으로 통해서 정확도를 최대한도로 높여 보세요.

**최소한 78.9% 이상을 얻으세요.**

In [52]:
deep_feature_model_20 = graphlab.logistic_classifier.create(train_data, target='label', features=['deep_features'], validation_set=None, max_iterations=20)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

WARNING: Detected extremely low variance for feature(s) 'deep_features' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

Number of examples          : 2005

...

| 19        | 26       | 0.250000  | 54.224225    | 0.957107          |

| 20        | 27       | 0.250000  | 56.211357    | 0.958603          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [53]:
deep_feature_model_20.evaluate(test_data)['accuracy']

0.789

#### (참고) evaluate 메소드로 정확도를 구하다가 에러가 나는 경우 아래의 구문을 이용하세요.

    corrects = (model.predict(image_test) == image_test['label']).sum()
    print "Accuracy: ", corrects/float(len(image_test))

# Programming Assignment

**이하의 내용은 'W6b - Deep Features for Image Retrieval'를 마친 후에 진행하세요.**

## Quiz1
학습 데이터에 가장 적은 카테고리는 무엇인가요?

    SArray.sketch_summary()

In [54]:
train_data['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


#### (대안) sketch_summary를 사용하지 않는 방법

In [55]:
label_dict = {}
for row in train_data:
    label = row['label']
    if label in label_dict:
        label_dict[label] += 1
    else:
        label_dict[label] = 1
print(label_dict)

{'automobile': 509, 'dog': 509, 'bird': 478, 'cat': 509}


# 특정 카테고리만을 위한 이미지 조회 모델 만들기 

각각의 레이블만을 위한 이미지 조회 모델을 만들고자 합니다. 이를 위해서 학습 데이터를 4가지 레이블별로 나눈 다음, 나눠진 해당 데이터만을 이용해서 각 레이블별 모델을 학습니다. 이렇게 하면 학습 시간을 현저히 줄일 수 있습니다. 

## 각 레이블별 학습 데이터 세트 만들기

필터링을 이용해서 4개의 레이블별 학습 데이터 세트를 만듭니다.

1. 4개 학습데이터를 개별적으로 만들기 (ex. image_train_dog, image_train_cat)
2. for-loop를 이용해서 4개의 학습 데이터를 하나의 dictionary에 레이블을 key로, 데이터를 value로 저장하면 됩니다.

이하에서는 2번째 방법을 사용하고자 합니다. 어려운 경우 1번 방법을 사용하세요.

In [56]:
## key로 사용할 레이블 뽑기. 직접 만들어도 됩니다.
labels = label_dict.keys()
print(labels)

['automobile', 'dog', 'bird', 'cat']


### 4개의 레이블별 학습 데이터 세트 만들어서 dictionary에 저장하기

- 먼저 사용하고자 하는 dictionary를 먼저 선언합니다. dictionary는 사용 전에 선언이 되어 있지 않으면 에러가 납니다. 이때 변수명은 **image_train_for_specific_label**을 사용합시다.
        

- 그런 다음 for-loop를 이용해서 각 레이블별로
    - 전체 학습 데이터 중에 해당 레이블에 해당하는 학습 데이터만을 골라 내고,
    - 그 결과값을 새로운 dictionary에 해당 레이블을 key로, 데이터 세트를 value로 저장합니다.

    dictionary[key] = value

In [58]:
image_train_for_specific_label = {}

for i in labels:
    image_train_for_specific_label[i] = train_data[train_data['label']== i]

dictionary를 출력해서 잘 만들어졌는지 확인해 봅시다.

In [59]:
print(image_train_for_specific_label)

{'automobile': Columns:
	id	int
	image	Image
	label	str
	deep_features	array
	image_array	array
	image_array_norm	array

Rows: 509

Data:
+------+----------------------+------------+-------------------------------+
|  id  |        image         |   label    |         deep_features         |
+------+----------------------+------------+-------------------------------+
|  97  | Height: 32 Width: 32 | automobile | [1.57818555832, 0.0, 0.0, ... |
| 136  | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.0, 0.0, ... |
| 302  | Height: 32 Width: 32 | automobile | [0.583938002586, 0.0, 0.0,... |
| 312  | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.39282339... |
| 323  | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 4.42310428... |
| 536  | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.0, 0.0, ... |
| 593  | Height: 32 Width: 32 | automobile | [1.65033948421, 0.0, 0.0, ... |
| 962  | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.39552795... |
| 997  | Height

너무 많아서 확인이 어렵습니다. dictionary에 dog을 key값으로 설정해서 출력해 봅시다.

In [60]:
image_train_for_specific_label['dog']

id,image,label,deep_features,image_array
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
177,Height: 32 Width: 32,dog,"[0.0, 1.45965671539, 0.0,0.422992348671, 0.0, ...","[55.0, 75.0, 42.0, 51.0,76.0, 37.0, 57.0, 83.0, ..."
424,Height: 32 Width: 32,dog,"[0.942399680614, 0.0,0.220352768898, 0.0, ...","[60.0, 35.0, 18.0, 63.0,49.0, 38.0, 66.0, 56.0, ..."
462,Height: 32 Width: 32,dog,"[1.43462562561, 0.0, 0.0,0.0, 0.0, 0.0, ...","[86.0, 69.0, 75.0, 57.0,41.0, 48.0, 46.0, 35.0, ..."
542,Height: 32 Width: 32,dog,"[0.451547086239, 0.0,0.0, 0.0, 0.0, 0.0, ...","[196.0, 174.0, 113.0,140.0, 117.0, 65.0, 8 ..."
573,Height: 32 Width: 32,dog,"[0.592360973358, 0.0,0.0, 0.0, 0.0, 0.0, ...","[153.0, 103.0, 52.0,151.0, 102.0, 49.0, ..."
851,Height: 32 Width: 32,dog,"[0.690123438835, 0.0,0.0, 0.0, 0.305860161 ...","[39.0, 6.0, 4.0, 53.0,23.0, 24.0, 57.0, 37.0, ..."
919,Height: 32 Width: 32,dog,"[0.0, 0.177558660507,0.139396846294, 0.0, ...","[29.0, 43.0, 4.0, 24.0,35.0, 6.0, 24.0, 37.0, ..."
1172,Height: 32 Width: 32,dog,"[0.517601490021, 0.0,1.96418333054, 0.0, 0.0, ...","[182.0, 180.0, 197.0,196.0, 192.0, 209.0, ..."


## 레이블별 전문 모델 만들기
이제 각 레이블별 전문 모델을 만들어야 합니다. 위에서와 마찬가지로 2가지 방법이 있습니다.

1. 4개 모델을 개별적으로 만들기 (ex. dog_model, cat_model)
2. 각 모델을 for-loop를 이용해서 만들면서 하나의 dictionary에 저장하기. 레이블을 key로, 모델을 value로 저장하면 됩니다.

이하에서는 2번째 방법을 사용하고자 합니다. 어려운 경우 1번 방법을 사용하세요.

### 4개의 레이블별 전문 모델 만들어서 dictionary에 저장하기

- 먼저 사용하고자 하는 dictionary를 먼저 선언합니다. dictionary는 사용 전에 선언이 되어 있지 않으면 에러가 납니다. 이때 변수명은 **label_specific_models**을 사용합시다.
        
        
- 그런 다음 for-loop를 이용해서 각 레이블별로
    - 근접 이웃을 구하는 모델을 학습합니다.
    
            graphlab.nearest_neighbors.create(학습데이터, features=사용하고자하는피쳐리스트, label=레이블컬럼이름)
        
    - 학습된 모델을 새로운 dictionary에 해당 레이블을 key로, 모델을 value로 저장합니다.

In [61]:
label_specific_models = {}

for i in labels:
    label_specific_models[i] = graphlab.nearest_neighbors.create(image_train_for_specific_label[i], features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

## Quiz 2

아래 이미지 중에서 테스트 데이터의 첫번째 이미지 (image_test[0:1])에 가장 가까운 학습 데이터의 'cat' 레이블 이미지는 어느 것인가요?

참고: 퀴즈에서 보기로 주어지는 이미지를 참고하세요.

In [62]:
def get_images_from_ids(query_result):
    return train_data.filter_by(query_result['reference_label'],'id')

In [63]:
get_images_from_ids(label_specific_models['cat'].query(test_data[0:1]))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 40.35ms      |

| Done         |         | 100         | 175.002ms    |

+--------------+---------+-------------+--------------+

## Quiz 3

아래 이미지 중에서 테스트 데이터의 첫번째 이미지 (image_test[0:1])에 가장 가까운 학습 데이터의 'dog' 레이블 이미지는 어느 것인가요?

참고: 퀴즈에서 보기로 주어지는 이미지를 참고하세요.

In [175]:
get_images_from_ids(label_specific_models['dog'].query(test_data[0:1]))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 34.4ms       |

| Done         |         | 100         | 189.548ms    |

+--------------+---------+-------------+--------------+

## Quiz 4

테스트 데이터의 첫번째 이미지를 기준으로 학습 데이터의 'cat' 이미지 중 5개-근접 이웃들과의 평균적인 거리는 얼마인가요?

In [66]:
label_specific_models['cat'].query(test_data[0:1])['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 54.305ms     |

| Done         |         | 100         | 230.461ms    |

+--------------+---------+-------------+--------------+

36.15573070978294

## Quiz 5

테스트 데이터의 첫번째 이미지를 기준으로 학습 데이터의 'dog' 이미지 중 5개-근접 이웃들과의 평균적인 거리는 얼마인가요?

In [67]:
label_specific_models['dog'].query(test_data[0:1])['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 74.85ms      |

| Done         |         | 100         | 224.914ms    |

+--------------+---------+-------------+--------------+

37.77071136184157

## Quiz 6 
 
'cat' 데이터의 5개-근접 이웃들 (5-nearest neighbors)과 'dog' 데이터의 5개-근접 이웃들 중에서 테스트 데이터의 첫번째 이미지가 평균적으로 어느 쪽에 더 가까울까요?

In [68]:
#cat

### (중급) 답을 출력하는 코드 만들기

한 줄의 코드를 이용해서 답을 아래와 같이 출력하는 코드를 만들어 봅시다. 
- Dog 데이터가 더 가깝다.
- Cat 데이터가 더 가깝다.

이를 위해서는 아래의 inline if-else를 사용해야 합니다.

#### inline if-else 사용하기

아래와 같이 한줄로 if-else 구문을 만들어서 사용할 수 있습니다.

    true인경우값 if (조건문) else false인경우값 

아래와 같은 구문을 실행하면 True가 출력됩니다. 직접 입력해 보세요.

    print "True" if 1 < 2 else "False"

In [78]:
if label_specific_models['dog'].query(test_data[0:1])['distance'].mean() < label_specific_models['cat'].query(test_data[0:1])['distance'].mean():
    print 'Dog 데이터가 더 가깝다'
else:
    print 'Cat 데이터가 더 가깝다'

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 43.2ms       |

...

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 30.789ms     |

| Done         |         | 100         | 145.215ms    |

Cat 데이터가 더 가깝다


+--------------+---------+-------------+--------------+

## Quiz 7

테스트 세트의 'dog' 이미지를 구분하는 최근접 이웃 구분자 (1-nearest neighbor classifier)의 정확도는 얼마 정도인가요?

# 도전 과제

이하에서는 하나의 레이블로만 이루어진 테스트 데이터를 이용해서 위에서 만든 레이블별 전문 모델들이 얼마의 정확도를 가지는지를 측정해 보고자 합니댜. 상식적으로 생각해 볼 때 dog 이미지를 이용해서 학습한 모델이 테스트로 주어지는 dog 이미지에 대해서 가장 좋은 성능을 보여야 합니다. 이를 확인하는 것이 이 과제의 내용입니다.

- 'dog' 레이블로만 이루어진 테스트 데이터를 만든다.
- 위에서 만든 레이블별 전문 모델에 해당 테스트 데이터를 적용해서 최근접 이웃을 구한다.
- 구해진 최근접 이웃이 'dog' 레이블에 해당하는지를 확인해서 정확도를 구한다.


참고: 이하는 처음에는 헷갈릴 수 있는 내용이지만 실제로는 4개의 변수를 하나의 dictionary에 넣어서 하나의 변수명으로 사용하고자 하는 단순한 목적을 위한 코드가 대부분입니다. 몇번 해 보면 익숙해질 내용이라서 반복적으로 따라 할 수 있도록 과제를 구성했습니다. 혹시라도 너무 어렵게 느껴진다면 1번 방법 - 개별적인 변수로 저장해서 사용해도 - 으로도 먼저 과제를 푼 다음에 도전해 보세요.


## 각 레이블별 테스트 데이터 세트 만들기

먼저 필터링을 이용해서 4개의 레이블별 테스트 데이터 세트를 만듭니다.

1. 4개 테스트 데이터를 개별적으로 만들기 (ex. image_test_dog, image_test_cat)
2. for-loop를 이용해서 4개의 학습 데이터를 하나의 dictionary에 저장하기. 레이블을 key로, 데이터를 value로 저장하면 됩니다.

이하에서는 2번째 방법을 사용하고자 합니다. 어려운 경우 1번 방법을 사용하세요.

### 4개의 레이블별 데이터 세트 만들어서 dictionary에 저장하기

- 먼저 사용하고자 하는 dictionary를 먼저 선언합니다. 선언이 되지 않으면 에러가 납니다. 이때 변수명은 **image_test_for_specific_label**을 사용합시다.
        

- 그런 다음 for-loop를 이용해서 각 레이블별로
    - 전체 테스트 데이터 중에 해당 레이블에 해당하는 테스트 데이터만을 골라 내고,
    - 그 결과값을 새로운 dictionary에 해당 레이블을 key로, 데이터 세트를 value로 저장합니다.

    dictionary[key] = value

In [86]:
image_test_for_specific_label = {}

for i in labels:
    image_test_for_specific_label[i] = test_data[test_data['label']==i]

4개의 데이터 세트를 저장한 dict를 출력해 봅시다.

In [87]:
image_test_for_specific_label

{'automobile': Columns:
 	id	int
 	image	Image
 	label	str
 	deep_features	array
 	image_array	array
 	image_array_norm	array
 
 Rows: 1000
 
 Data:
 +-----+----------------------+------------+-------------------------------+
 |  id |        image         |   label    |         deep_features         |
 +-----+----------------------+------------+-------------------------------+
 |  6  | Height: 32 Width: 32 | automobile | [0.23135882616, 0.0, 0.0, ... |
 |  9  | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.0, 0.0, ... |
 |  37 | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.0, 0.0, ... |
 |  66 | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.0, 0.0, ... |
 |  81 | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.60016828... |
 |  82 | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.97667396... |
 | 104 | Height: 32 Width: 32 | automobile | [0.682431340218, 0.5678311... |
 | 105 | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 3.45144057... |
 | 1

'cat' 레이블에 해당하는 데이터 세트 (SFrame)에 대해서 show 메소드를 실행해 봅시다.

In [88]:
image_test_for_specific_label['cat'].head()

id,image,label,deep_features,image_array
0,Height: 32 Width: 32,cat,"[1.13469004631, 0.0, 0.0,0.0, 0.0366497635841, ...","[158.0, 112.0, 49.0,159.0, 111.0, 47.0, ..."
8,Height: 32 Width: 32,cat,"[0.0, 0.0,0.0344192385674, 0.0, ...","[23.0, 19.0, 23.0, 19.0,21.0, 28.0, 21.0, 16.0, ..."
46,Height: 32 Width: 32,cat,"[0.0, 0.0,0.255758941174, 0.0, ...","[22.0, 27.0, 19.0, 30.0,36.0, 26.0, 30.0, 36.0, ..."
53,Height: 32 Width: 32,cat,"[0.61073166132, 0.0, 0.0,0.0, 0.0, 0.0, ...","[255.0, 255.0, 251.0,249.0, 251.0, 246.0, ..."
61,Height: 32 Width: 32,cat,"[0.935939073563, 0.0,0.778302431107, 0.0, ...","[10.0, 10.0, 10.0, 10.0,10.0, 10.0, 10.0, 10.0, ..."
63,Height: 32 Width: 32,cat,"[0.0, 0.0,0.359475255013, 0.0, ...","[233.0, 225.0, 209.0,201.0, 170.0, 123.0, ..."
68,Height: 32 Width: 32,cat,"[0.708701610565, 0.0,0.0, 0.0, 0.198133170 ...","[61.0, 44.0, 21.0, 55.0,41.0, 17.0, 58.0, 48.0, ..."
77,Height: 32 Width: 32,cat,"[0.295638740063, 0.0,0.0, 0.0, 0.0, 0.0, ...","[51.0, 49.0, 53.0, 57.0,56.0, 61.0, 61.0, 61.0, ..."
78,Height: 32 Width: 32,cat,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 8.64840698242, 0.0, ...","[38.0, 24.0, 12.0, 50.0,31.0, 24.0, 37.0, 21.0, ..."
91,Height: 32 Width: 32,cat,"[1.24886679649, 0.0, 0.0,0.0196405649185, ...","[72.0, 75.0, 57.0, 70.0,74.0, 52.0, 63.0, 67.0, ..."


'cat' 레이블 데이터로만 구성된 SFrame의 내용이 화면에 출력되어야 합니다.

## dog 레이블로만 구성된 테스트 데이터를 이용한 거리 구하기

이제 image_test_dog 데이터 - 테스트 데이터 중에서 dog 레이블로만 구성된 서브 데이터 세트 - 만을 이용한 각 레이블별 모델의 거리를 구하고자 합니다. 위에서와 마찬가지로 2가지 방법이 있습니다.

1. 4개 거리를 계산한 모델을 개별적으로 구해서 저장하기 (ex. dog_dog_neighbors, dog_cat_neighbors)
2. 각 모델을 for-loop를 이용해서 만들면서 하나의 dictionary에 저장하기. 레이블을 key로, 모델을 value로 저장하면 됩니다.

이하에서는 2번째 방법을 사용하고자 합니다. 어려운 경우 1번 방법을 사용하세요.

### 4개의 레이블 별 데이터 세트 만들어서 dictionary에 저장하기

- 먼저 사용하고자 하는 dictionary를 먼저 선언합니다. 선언이 되지 않으면 에러가 납니다. 이때 변수명은 **dog_distance_for_specific_label**을 사용합시다.
    
        
    
- 그런 다음 for-loop를 이용해서 각 레이블별로
    - dog 레이블로만 구성된 테스트 데이터를 해당 레이블의 모델에 적용해서 최근접 이웃을 구하고
    - 이 결과값을 새로운 dictionary에 해당 레이블을 key로, 결과값을 value로 저장합니다.
    
    model.query(학습데이터, k=1)
    
    dictionary[key] = value

최근접 이웃을 구하고자 하므로 k는 1이 됩니다.

In [89]:
dog_distance_for_specific_label = {}

for i in labels:
    dog_distance_for_specific_label[i] = label_specific_models[i].query(image_test_for_specific_label['dog'], k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 2

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 255000  | 50.0982     | 1.01s        |

| Done         | 509000  | 100         | 1.02s        |

+--------------+---------+-------------+--------------+

새롭게 만들어진 dictionary의 key로 'dog'을 넣어서 화면에 출력해 봅시다.

In [90]:
dog_distance_for_specific_label['dog']

query_label,reference_label,distance,rank
0,49803,33.4773590373,1
1,5755,32.8458495684,1
2,20715,35.0397073189,1
3,13387,33.9010327697,1
4,12089,37.4849250909,1
5,6094,34.945165344,1
6,3431,39.0957278345,1
7,6184,37.7696131032,1
8,2167,35.1089144603,1
9,7776,43.2422832585,1


SFrame이 출력되는 것을 확인할 수 있습니다. 여기에서 우리가 주목할 컬럼은 바로 distance입니다.

## 각 모델의 최근접 이웃과의 거리로 이루어진 SFrame 만들기

위에서 출력한 SFrame은 'dog' 레이블로 학습된 모델에 'dog' 레이블로만 구성된 테스트 데이터를 적용했을 때 얻어진 최근접 이웃과 해당 이미지 간의 거리를 'distance' 컬럼에 저장한 데이터 구조입니다. 

우리는 'dog' 레이블로만 구성된 테스트 데이터를 4개 모델 전체에 적용해서 얻어진 최근접 이웃과의 거리를 비교해서 'dog' 레이블로 학습된 모델에서 최단 거리가 나오는지를 확인하고자 합니다.


### dog_distance 구하기

- 먼저 새로 만들고자 하는 SFrame을 선언합니다. 아래 구문을 이용해서 새로운 SFrame을 선언할 수 있습니다. 이때 **dog_distances**라는 변수명을 사용하세요
    
    graphlab.SFrame()
            
    
- 그런 다음 for-loop를 이용해서 각 레이블별로
    - 해당 레이블의 dog_distance_for_specific_label에서 'distance' 컬럼만을 뜯어 내고
    - 이 결과값을 새로운 SFrame의 하나의 컬럼으로 저장합니다.
        - 이때 해당 컬럼의 이름은 'dog-dog', 'dog-cat', 'dog-automobile', 'dog-bird'와 같이 만듭니다.

In [91]:
#참고: 아래 코드를 실행해 본 다음에 응용해서 이 문제를 푸세요. dictionary의 key값은 string이어야 합니다.

dict_sample = {}
dict_sample["ke" + "y"] = 3
print dict_sample

{'key': 3}


In [140]:
dog_distance = graphlab.SFrame()

for i in labels:
    dog_distance['dog-'+i] = dog_distance_for_specific_label[i]['distance']

새롭게 만들어진 dog_distance를 출력해 보세요.

In [141]:
dog_distance

dog-automobile,dog-dog,dog-bird,dog-cat
41.9579761457,33.4773590373,41.7538647304,36.4196077068
46.0021331807,32.8458495684,41.3382958925,38.8353268874
42.9462290692,35.0397073189,38.6157590853,36.9763410854
41.6866060048,33.9010327697,37.0892269954,34.5750072914
39.2269664935,37.4849250909,38.272288694,34.778824791
40.5845117698,34.945165344,39.1462089236,35.1171578292
45.1067352961,39.0957278345,40.523040106,40.6095830913
41.3221140974,37.7696131032,38.1947918393,39.9036867306
41.8244654995,35.1089144603,40.1567131661,38.0674700168
45.4976929401,43.2422832585,45.5597962603,42.7258732951


'dog' 이미지로만 구성된 테스트 데이터를 4개의 개별 모델에 적용했을 때 구해진 최근접 이웃과의 거리가 저장된 데이터 세트를 얻었습니다.

이제 첫번째 데이터의 거리들만을 출력해 보세요.

In [155]:
dog_distance[0]

{'dog-automobile': 41.957976145712024,
 'dog-bird': 41.753864730351246,
 'dog-cat': 36.41960770675437,
 'dog-dog': 33.47735903726335}

## dog 이미지가 맞는지 확인하기

이제 apply를 이용해서 각 줄의 데이터가 dog 이미지가 맞는지를 판별하고자 합니다. 이를 위해 함수를 하나 만들어야 합니다.

### 개 이미지가 맞는 확인하는 함수 만들기

- 함수 이름은 **is_dog_correct**를 사용합시다.
- SFrame.apply를 사용하기 때문에 인자로는 SFrame의 한줄을 받게 됩니다.
- for-loop를 이용해서 각 컬럼값 별로
    - 해당 컬럼의 값이 dog-dog 컬럼의 값보다 작다면 False를 return하고 종료합니다.
    - 그렇지 않다면 계속 for-loop를 실행합니다.
- 만약 for-loop가 False를 반환하지 않고 끝났다면 True를 return합니다.

In [163]:
def is_dog_correct(row):
    temp = ['dog-dog', 'dog-cat', 'dog-bird', 'dog-automobile']
    for i in temp:
        if row[i] < row['dog-dog']:
            result = False
            break
        else:
            result = True
    return result

새롭게 만든 is_dog_correct 함수가 잘 작동하는지를 아래의 구문으로 확인해 봅시다.

In [164]:
is_dog_correct(dog_distance[0])

True

원하는 결과 나왔나요?

### dog 이미지가 맞는지 정확도 구하기

dog_distances에 새롭게 만든 함수를 적용해서 dog 이미지가 맞는지를 판별하고 이렇게 구한 결과가 얼마의 정확도를 가지는지를 구해 봅시다.

- dog_distances에 apply와 is_dog_correct를 적용합시다.
- 위의 결과값은 어떤 유형인가요? 어떻게 하면 개수를 셀 수 있을까요?
- 나누기의 분모는 무엇이어야 할까요?
- 나누기할 때 반드시 float()를 사용하세요.


In [165]:
dog_distance.apply(is_dog_correct)

dtype: int
Rows: 1000
[1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, ... ]

In [166]:
type(dog_distance.apply(is_dog_correct))

graphlab.data_structures.sarray.SArray

In [167]:
dog_distance.apply(is_dog_correct).sum() / float(len(dog_distance))

0.678

결과값은 0.678이 나와야 합니다.

## Quiz

### 고양이 이미지가 맞는지 정확도 구하기

위와 동일한 방법으로 고양이 이미지에 대해서도 구해 봅시다

- 먼저 cat_distance_for_specific_label을 구합니다.
- 그 다음 cat_distances를 구합니다.
- is_cat_correct 함수를 만듭니다.
- apply를 이용해서 고양이 테스트 이미지에 적용한 다음 정확도를 구합니다.

In [168]:
cat_distance_for_specific_label = {}

for i in labels:
    cat_distance_for_specific_label[i] = label_specific_models[i].query(image_test_for_specific_label['cat'], k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 2

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 255000  | 50.0982     | 1.11s        |

| Done         | 509000  | 100         | 1.16s        |

In [170]:
cat_distance = graphlab.SFrame()

for i in labels:
    cat_distance['cat-'+i] = cat_distance_for_specific_label[i]['distance']

In [173]:
def is_cat_correct(row):
    temp = ['cat-dog', 'cat-cat', 'cat-bird', 'cat-automobile']
    for i in temp:
        if row[i] < row['cat-cat']:
            result = False
            break
        else:
            result = True
    return result

In [174]:
cat_distance.apply(is_cat_correct).sum() / float(len(cat_distance))

0.548

0.548의 결과값이 나와야 합니다.